In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import datetime
import logging
import warnings
from pathlib import Path 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb 
import lightgbm as lgb 
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')
# 数据集目录
data_path = Path(r'/Users/liuzhi/datasets/tc_金融风控-贷款违约预测')

In [3]:
train = pd.read_csv(f"{data_path}/train.csv")
testA = pd.read_csv(f"{data_path}/testA.csv")
logging.info(f"train shape: {train.shape}")
logging.info(f"testA shape: {testA.shape}")
train.head()

2021-01-12 17:33:51,537 INFO: train shape: (800000, 47)
2021-01-12 17:33:51,538 INFO: testA shape: (200000, 46)


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


### 特征预处理

In [4]:
# Fucntion from EDA
def split_numerical_and_category_features(df):
    r'''拆分数值列、类别列'''
    numerical_features = list(df.select_dtypes(exclude=['object']).columns)
    category_features = list(filter(lambda x: x not in numerical_features, list(df.columns)))
    return numerical_features, category_features

def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [5]:
# 查找出数据中的对象特征和数值特征
numerical_features, category_featrues = split_numerical_and_category_features(train)
label = 'isDefault'
numerical_features.remove(label)

### 缺失值填充
1. 所有缺失值替换为指定值，如 0
    data = data.fillna(0)

2. 用缺失值上面的值替换缺失值
    data = data.fillna(axis=0, method='ffill')

3. 纵向用缺失值下面的值替换缺失值，且设置最多填充2个连续的缺失值
    data = data.fillna(axis=0, method='bfill', limit=2)

In [6]:
# 查看缺失值情况
train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           1
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n3                  

In [7]:
# 按平均值填充数值类型特征
train[numerical_features] = train[numerical_features].fillna(train[numerical_features].median())
testA[numerical_features] = testA[numerical_features].fillna(testA[numerical_features].median())

# 按众数填充类别特征
train[category_featrues] = train[category_featrues].fillna(train[category_featrues].mode())
testA[category_featrues] = testA[category_featrues].fillna(testA[category_featrues].mode())

In [8]:
# 再次查看填充缺失值后的情况
train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           0
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  0
regionCode                0
dti                       0
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies        0
revolBal                  0
revolUtil                 0
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     0
policyCode                0
n0                        0
n1                        0
n2                        0
n3                  

In [9]:
# 查看类别特征
category_featrues

['grade', 'subGrade', 'employmentLength', 'issueDate', 'earliesCreditLine']

In [10]:
# issueDate为时间格式的特征

# 转换为时间格式
for data in [train,testA]:
    data['issueDate'] = pd.to_datetime(data['issueDate'], format=r'%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', r'%Y-%m-%d')
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [11]:
train['employmentLength'].value_counts(dropna=False).sort_index()

1 year        52489
10+ years    262753
2 years       72358
3 years       64152
4 years       47985
5 years       50102
6 years       37254
7 years       35407
8 years       36192
9 years       30272
< 1 year      64237
NaN           46799
Name: employmentLength, dtype: int64

In [12]:
# 对象类型特征转换到数值
for data in [train, testA]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace(to_replace='< 1 year', value='0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [13]:
train['employmentLength'].value_counts(dropna=False).sort_index()

0.0      64237
1.0      52489
2.0      72358
3.0      64152
4.0      47985
5.0      50102
6.0      37254
7.0      35407
8.0      36192
9.0      30272
10.0    262753
NaN      46799
Name: employmentLength, dtype: int64

In [14]:
# 对 earliesCreditLine 进行处理
train['earliesCreditLine'].sample(5)

501357    Mar-2010
505033    Oct-2013
514501    Nov-1986
480706    Sep-2002
379293    Nov-1997
Name: earliesCreditLine, dtype: object

In [15]:
for data in [train, testA]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

### 类别特征处理

In [17]:
# 部分类别特征
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', 'applicationType', 'initialListStatus', 'title', 'policyCode']
for f in cate_features:
    print(f, '类型数：', data[f].nunique())

grade 类型数： 7
subGrade 类型数： 35
employmentTitle 类型数： 79282
homeOwnership 类型数： 6
verificationStatus 类型数： 3
purpose 类型数： 14
postCode 类型数： 889
regionCode 类型数： 51
applicationType 类型数： 2
initialListStatus 类型数： 2
title 类型数： 12058
policyCode 类型数： 1


In [18]:
# 像等级这种类别的特征，具有优先级的可以用labelencode或者自映射
for data in [train, testA]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})

In [19]:
# 类目数大于2的，又不是高维稀疏的，且纯分类特征
for data in [train, testA]:
    data = pd.get_dummies(data, columns=['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'])

### 异常值处理

发现异常值后，一定要先分清是什么原因导致的异常值，然后再考虑如何处理。首先，如果这一异常值并不代表一种规律性的，而是极其偶然的现象，或者说你并不想研究这种偶然的现象，这时可以将其删除。其次，如果异常值存在且代表了一种真实存在的现象，那就不能随便删除。在现有的欺诈场景中很多时候欺诈数据本身相对于正常数据勒说就是异常的，我们要把这些异常点纳入，重新拟合模型，研究其规律。能用监督的用监督模型，不能用的还可以考虑用异常检测的算法来做。

In [20]:
# 检测异常的方法一： 均方差

def find_outliers_by_3segama(data, fea):
    data_std = np.std(data[fea])
    data_mean = np.mean(data[fea])
    outliers_cut_off = data_std*3
    lower_rule = data_mean - outliers_cut_off
    upper_rule = data_mean + outliers_cut_off
    data[f"{fea}_outliers"] = data[fea].apply(lambda x: str('异常值') if x>upper_rule or x<lower_rule else '正常值')
    return data 

In [22]:
# 得到特征的异常值之后，可以进一步分析变量异常值和目标变量的关系
data_train = train.copy()
for fea in numerical_features:
    data_train = find_outliers_by_3segama(data_train, fea)
    print(data_train[f"{fea}_outliers"].value_counts())
    print(data_train.groupby(f"{fea}_outliers")['isDefault'].sum())
    print("*"*10)

正常值    800000
Name: id_outliers, dtype: int64
id_outliers
正常值    159610
Name: isDefault, dtype: int64
**********
正常值    800000
Name: loanAmnt_outliers, dtype: int64
loanAmnt_outliers
正常值    159610
Name: isDefault, dtype: int64
**********
正常值    800000
Name: term_outliers, dtype: int64
term_outliers
正常值    159610
Name: isDefault, dtype: int64
**********
正常值    794259
异常值      5741
Name: interestRate_outliers, dtype: int64
interestRate_outliers
异常值      2916
正常值    156694
Name: isDefault, dtype: int64
**********
正常值    792046
异常值      7954
Name: installment_outliers, dtype: int64
installment_outliers
异常值      2152
正常值    157458
Name: isDefault, dtype: int64
**********
正常值    800000
Name: employmentTitle_outliers, dtype: int64
employmentTitle_outliers
正常值    159610
Name: isDefault, dtype: int64
**********
正常值    799701
异常值       299
Name: homeOwnership_outliers, dtype: int64
homeOwnership_outliers
异常值        62
正常值    159548
Name: isDefault, dtype: int64
**********
正常值    793973
异常值      

In [23]:
# 删除异常值
for fea in numerical_features:
    data_train = data_train[data_train[f"{fea}_outliers"] == '正常值']
    data_train = data_train.reset_index(drop=True)

In [24]:
# 检测异常的方法二： 箱型图
# 四分位数会将数据分为三个点和四个区间，IQR = Q3 -Q1，下触须=Q1 − 1.5x IQR，上触须=Q3 + 1.5x IQR；

### 数据分桶

In [25]:
# 1. 固定宽度分箱
# 当数值横跨多个数量级时，最好按照 10 的幂（或任何常数的幂）来进行分组：0~9、10~99、100~999、1000~9999，等等。固定宽度分箱非常容易计算，但如果计数值中有比较大的缺口，就会产生很多没有任何数据的空箱子。

# 通过除法映射到间隔均匀的分箱中，每个分箱的取值范围都是loanAmnt/1000
data['loanAmnt_bin1'] = np.floor_divide(data['loanAmnt'], 1000)

# 通过对数函数映射到指数宽度分箱
data['loanAmnt_bin2'] = np.floor(np.log10(data['loanAmnt']))

In [26]:
# 2. 分位数分箱
data['loanAmnt_bin3'] = pd.qcut(data['loanAmnt'], 10, labels=False)

In [27]:
# 3. 卡方分箱及其他分箱方法的尝试

### 特征交互

In [28]:
for col in ['grade', 'subGrade']:
    temp_dict = train.groupby([col])['isDefault'].agg(['mean']).reset_index().rename(columns={'mean': f"{col}_target_mean"})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[f"{col}_target_mean"].to_dict()

    train[f"{col}_target_mean"] = train[col].map(temp_dict)
    testA[f"{col}_target_mean"] = testA[col].map(temp_dict)

In [32]:
# 其他衍生变量mean和std
for df in [train, testA]:
    for item in ['n0','n1','n2','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14']:
        df[f'grade_to_mean_{item}'] = df['grade']/df.groupby([item])['grade'].transform('mean')
        df[f'grade_to_std_{item}'] = df['grade']/df.groupby([item])['grade'].transform('std')

### 特征编码

In [33]:
# label-encode 直接放入树模型中
# 高维类别特征需要进行转换
for col in tqdm(['employmentTitle', 'postCode', 'title', 'subGrade']):
    le = LabelEncoder()
    le.fit(list(train[col].astype(str).values)+list(testA[col].astype(str).values))
    train[col] = le.transform(list(train[col].astype(str).values))
    testA[col] = le.transform(list(testA[col].astype(str).values))

print('Label Encoding Finished!')

100%|██████████| 4/4 [00:09<00:00,  2.31s/it]Label Encoding Finished!



### 逻辑回归等模型需要单独增加的特征工程

对特征做归一化，去除相关性高的特征  
归一化目的是让训练过程更好更快的收敛，避免特征大吃小的问题  
去除相关性是增加模型的可解释性，加快预测过程。

### 特征选择

特征选择不是为了减少训练时间（实际上，一些技术会增加总体训练时间），而是为了减少模型评分时间。

特征选择的方法：

1 Filter  
    方差选择法  
    相关系数法（pearson 相关系数）  
    卡方检验  
    互信息法  

2 Wrapper （RFE）  
    递归特征消除法  
     
3 Embedded  
    基于惩罚项的特征选择法    
    基于树模型的特征选择  

In [ ]:
# 方差选择法
# 方差选择法中，先要计算各个特征的方差，然后根据设定的阈值，选择方差大于阈值的特征
from sklearn.feature_selection import VarianceThreshold

# 参数threshold为方差的阈值
VarianceThreshold(threshold=3).fit_transform(train, train_target)

In [ ]:
# 相关系数法
# Pearson 相关系数 皮尔森相关系数是一种最简单的，可以帮助理解特征和响应变量之间关系的方法，该方法衡量的是变量之间的线性相关性。 结果的取值区间为 [-1，1] ， -1 表示完全的负相关， +1表示完全的正相关，0 表示没有线性相关。
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
#选择K个最好的特征，返回选择特征后的数据
#第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，
#输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
#参数k为选择的特征个数
SelectKBest(k=5).fit_transform(train, train['isDefault'])

In [35]:
# 卡方检验

# 经典的卡方检验是用于检验自变量对因变量的相关性。 假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距。 其统计量如下： χ2=∑(A−T)2T，其中A为实际值，T为理论值
# (注：卡方只能运用在正定矩阵上，否则会报错Input X must be non-negative)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#参数k为选择的特征个数

SelectKBest(chi2, k=2).fit_transform(train, target_train)

In [ ]:
# 互信息法
from sklearn.feature_selection import SelectKBest
from minepy import MINE 
#由于MINE的设计不是函数式的，定义mic方法将其为函数式的，
#返回一个二元组，二元组的第2项设置成固定的P值0.5

def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)

# 参数k为选择的特征个数
SelectKBest(lambda X, Y: array(map(lambda x:mic(x, Y), X.T)).T, k=2).fit_transform(train, target_train)

In [ ]:
# Wrapper （Recursive feature elimination，RFE）
# 递归特征消除法 递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练。
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression

#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(train, target_train)

In [ ]:
# Embedded
# 基于惩罚项的特征选择法 使用带惩罚项的基模型，除了筛选出特征外，同时也进行了降维。
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
#带L1惩罚项的逻辑回归作为基模型的特征选择
SelectFromModel(LogisticRegression(penalty='l1', C=0.1)).fit_transform(train, target_train)

In [ ]:
# Embedded
# 基于树模型的特征选择 树模型中GBDT也可用来作为基模型进行特征选择。
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
SelectFromModel(GradientBoostingClassifier()).fit_transform(train, target_train)